In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Geração de dados
df = pd.read_csv('dados_eurusd_1_12_2023.csv')
df['action'] = np.random.choice(['buy', 'sell', 'hold'], size=len(data))
df['profit'] = np.random.uniform(-1, 1, len(data))

# Pré-processamento dos dados
scaler = StandardScaler()
df[['open', 'high', 'close', 'low']] = scaler.fit_transform(df[['open', 'high', 'close', 'low']])

# Divida os dados em treinamento e teste
train_data, test_data = train_test_split(df, test_size=0.2, shuffle=False)

# Prepare os dados para treinamento
def prepare_data(df, window_size):
    data = []
    actions = []
    profits = []
    for i in range(len(df) - window_size):
        window = df.iloc[i:i + window_size][['open', 'high', 'close', 'low']].values
        action = df.iloc[i + window_size]['action']
        profit = df.iloc[i + window_size]['profit']
        data.append(window)
        actions.append(action)
        profits.append(profit)
    return np.array(data), np.array(actions), np.array(profits)

# Configurações de tamanho
window_size = 8600  # Tamanho da janela de dados
batch_size = 16  # Tamanho do lote

# Prepare os dados
X_train, y_train, profits_train = prepare_data(train_data, window_size)
X_test, y_test, profits_test = prepare_data(test_data, window_size)

# Converta rótulos para representação one-hot
y_train_onehot = pd.get_dummies(y_train).values
y_test_onehot = pd.get_dummies(y_test).values

# Construa a CNN-GRU
model = tf.keras.Sequential([
    tf.keras.layers.Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(window_size, 4)),
    tf.keras.layers.GRU(32, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Treinamento usando janelas deslizantes com tamanho de lote menor
for i in range(window_size, len(X_train), batch_size):
    X_window_batch = X_train[i - window_size:i]
    y_label_batch = y_train_onehot[i - window_size:i]  # Ajuste aqui
    profit_batch = profits_train[i:i + batch_size]

    # Treinamento normal
    loss, accuracy = model.train_on_batch(X_window_batch, y_label_batch)
    print(f'Batch {i//batch_size + 1}/{len(X_train)//batch_size} - Loss: {loss:.4f} - Accuracy: {accuracy:.4f}')

    # Calcule a recompensa/punição com base no lucro
    for j in range(batch_size):
        if y_train[i + j] == 'buy':
            reward = profit_batch[j]  # Lucro ao comprar
        elif y_train[i + j] == 'sell':
            reward = -profit_batch[j]  # Perda ao vender
        else:
            reward = 0.0  # Nenhuma ação, nenhuma recompensa/punição

        # Atualize os pesos usando a recompensa calculada
        loss, accuracy = model.train_on_batch(X_window_batch[j:j + 1], y_label_batch[j:j + 1], sample_weight=np.array([reward]))

    # Libere a memória após o uso
    del X_window_batch, y_label_batch, profit_batch

# Avalie o modelo
accuracy = model.evaluate(X_test, y_test_onehot)[1]
print(f'Acurácia do modelo: {accuracy}')
